<a href="https://colab.research.google.com/github/masternobelphysics/astropytutorialsptbr/blob/main/(Jo%C3%A3o_Marcos_M_Ribeiro)Tradu%C3%A7%C3%A3o_de_Models_Quick_Fit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo 1: fazer um rápido ajuste usando astropy.modeling

## Autores
Rocio Kiman, Lia Corrales, Zé Vinícius, Kelle Cruz, Stephanie T. Douglas

#Tradução

João  Marcos Modesto Ribeiro

## Objetivo do aprendizado
* Uso do `astroquery`  para baixar os dados do Vizier.
* Uso de modelos basicos em `astropy.modeling´.
* Aprender funções comuns para o ajuste. 
* Gerar um ajuste rápido para os dados.
* Plotar o modelo com os dados data.
* Comparar diferentes modelos e fittros.

## Palavras chaves
modelagem, ajuste de modelo, astrostatística, astroquery, Vizier, scipy, matplotlib, barras de erro, gráficos de dispersão.

## Objetivo
Neste tutorial, nós iremos nos familiarizar com modelos disponiveis em  [astropy.modeling](http://docs.astropy.org/en/stable/modeling/ ) e aprendermos  como fazer um ajuste  rápido para nossos dados.

### Pacotes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting
from astroquery.vizier import Vizier
import scipy.optimize
# Fazer estrutura de plots nos notebooks
%matplotlib inline 

## 1) Ajustando um modelo linear: Três passos para ajustar dados usando astropy.modeling

Vamos começar com um **ajuste linear para dados reais**. Os dados vem de um paper [Bhardwaj et al. 2017](https://ui.adsabs.harvard.edu/?#abs/2017A%26A...605A.100B). Este é um catalogo de **Cefeidas Tipo II**, que é um tipo de **estrelas variáveis** que pulsam com um periodo entre 1 e 50 dias. Nesta  parte do tutorial, Nós vamos medir a relação do ** Periodo-Luminosidade das Cefeidas**  usando `astropy.modeling. Esta relação afirma que se uma estrela tem um período mais longo, a luminosidade que medimos é maior.

Para obtê-lo , nós vamos importar do [Vizier](http://vizier.u-strasbg.fr/viz-bin/VizieR) usando [astroquery](http://astroquery.readthedocs.io/en/latest/vizier/vizier.html).

In [ ]:
catalog = Vizier.get_catalogs('J/A+A/605/A100')

Este catálogo contém muitas informações, mas para este tutorial vamos trabalhar apenas com períodos e magnitudes. Vamos tomá-las usando as palvaras chaves `'Periodo'` and `__Ksmag__`.  Note que `'e__Ksmag_'` se refere às barras de erro nas medidas de magnitude.

In [ ]:
period = np.array(catalog[0]['Period']) 
log_period = np.log10(period)
k_mag = np.array(catalog[0]['__Ksmag_'])
k_mag_err = np.array(catalog[0]['e__Ksmag_'])

Vamos olhar as medidas de magnitude como função do período

In [ ]:
plt.errorbar(log_period, k_mag, k_mag_err, fmt='k.')
plt.xlabel(r'$\log_{10}$(Period [days])')
plt.ylabel('Ks')

Poderia-se dizer que existe uma relação linear  o periodo logaritimo  ae magnitudes. Para provar isso, nós queremos fazer um ajuste para os dados. This is where `astropy.modeling` é util. Nós vamos entender como em  três linhas simples , nós  podemos fazer qualquer ajuste que nós queremos. Nós vamos começar com o ajuste linear , mas primeiro, vamos entender o que o modelo e o ajustador são.

### Modelos em Astropy
[Modelos](http://docs.astropy.org/en/stable/modeling/#using-models) em Astropy are funções parametri8zadas conhecidas. Com este formato, eles asão faceis para definir e para usar, dados que não precisamos escrever a expressão da função todo tempo que queremos usar no modelo, justo o nome. Eles podem ser lineares ou não lineares nas variaveis. São alguns exemplos de modelos:

* [Gaussian1D](http://docs.astropy.org/en/stable/api/astropy.modeling.functional_models.Gaussian1D.html#astropy.modeling.functional_models.Gaussian1D)
* [Trapezoid1D](http://docs.astropy.org/en/stable/api/astropy.modeling.functional_models.Trapezoid1D.html#astropy.modeling.functional_models.Trapezoid1D)
* [Polynomial1D](http://docs.astropy.org/en/stable/api/astropy.modeling.polynomial.Polynomial1D.html#astropy.modeling.polynomial.Polynomial1D)
* [Sine1D](http://docs.astropy.org/en/stable/api/astropy.modeling.functional_models.Sine1D.html#astropy.modeling.functional_models.Sine1D)
* [Linear1D](http://docs.astropy.org/en/stable/api/astropy.modeling.functional_models.Linear1D.html#astropy.modeling.functional_models.Linear1D)
* A [lista](http://docs.astropy.org/en/stable/modeling/#module-astropy.modeling.functional_models) continua.

Ajustadores em Astropy

Ajustadores em Astropy são classes responsaveis por fazer o ajuste. Eles podem ser lineares ou não-lineares nos parãmetros (não na variavel, como os mopdelos). Alguns exemplos são:

* [LevMarLSQFitter()](http://docs.astropy.org/en/stable/api/astropy.modeling.fitting.LevMarLSQFitter.html#astropy.modeling.fitting.LevMarLSQFitter)    Algorimo de  Levenberg-Marquardt  e estatística de mínimos quadrados. 
* [LinearLSQFitter()](http://docs.astropy.org/en/stable/api/astropy.modeling.fitting.LinearLSQFitter.html#astropy.modeling.fitting.LinearLSQFitter)  Uma classe que executa um ajuste linear de mínimos quadrados.
* [SLSQPLSQFitter()](http://docs.astropy.org/en/stable/api/astropy.modeling.fitting.SLSQPLSQFitter.html#astropy.modeling.fitting.SLSQPLSQFitter)        Algoritimo de otimização SLSQP e estatística de mínimos quadrados.
* [SimplexLSQFitter()](http://docs.astropy.org/en/stable/api/astropy.modeling.fitting.SimplexLSQFitter.html#astropy.modeling.fitting.SimplexLSQFitter)     algoritimo Simplex  e estatistica de minimos quadrados  .

* Mais detailhes [aqui](http://docs.astropy.org/en/stable/modeling/#id21)

Agora vamos  continuar com nosso ajuste.

#### Passo 1: Modelo

Primeiro, precisamos escolher qual modelo vamos usar para ajustar nossos dados. Como dissemos antes, nossos dados parecem uma relação linear, então vamos usar um modelo linear. 

In [ ]:
model = models.Linear1D()

#### Passo 2: Ajustador

Em segundo lugar, vamos escolher o ajustador que queremos usar. Esta escolha é basicamente qual método queremos usar para ajustar o modelo aos dados. Neste caso, nós vamos usar o [Linear Least Square Fitting](https://www.mathworks.com/help/curvefit/least-squares-fitting.html). No proximo Exerciciso ([Modelo 2: Criando um Modelo de Usuario de Definido](http://learn.astropy.org/rst-tutorials/User-Defined-Model.html)) Nós vamos analisar como escolher o ajustador. 

In [ ]:
fitter = fitting.LinearLSQFitter() 

#### Passo 3: Ajuste de dados 

Por fim, entregamos ao nosso **ajustador** (método para ajustar os dados) o **modelo** e os **dados** para realizar o ajuste. Observe que estamos incluindo pesos: Isso significa que valores com maior erro terão menor peso (menor importância) no ajuste, e o contrário para dados com erros menores. Essa forma de encaixe é chamada *Weighted Linear Least Squares* e vovê pode encontar  mais  informações sobre isto [aqui](https://www.mathworks.com/help/curvefit/least-squares-fitting.html) or [aqui](https://en.wikipedia.org/wiki/Least_squares#Weighted_least_squares).

In [ ]:
best_fit = fitter(model, log_period, k_mag, weights=1.0/k_mag_err**2)
print(best_fit)

E é isso !

Nós podemos avaliar  o ajuste do eixo x em  particular pelo `best_fit(x)`.

In [ ]:
plt.errorbar(log_period,k_mag,k_mag_err,fmt='k.')
plt.plot(log_period, best_fit(log_period), color='g', linewidth=3)  
plt.xlabel(r'$\log_{10}$(Period [days])')
plt.ylabel('Ks')

**Conclusão:** Lembre, que você pode fazer um ajusate de dedos com tres linhas de codigo:

1) Escolha um [modelo](http://docs.astropy.org/en/stable/modeling/#module-astropy.modeling.functional_models).

2) Escolha o [ajustador](http://docs.astropy.org/en/stable/modeling/#id21).

3)Passe para o ajustador o modelo e os dados para realizar o ajuste.

## Exercicio

Use o modelo `Polynomial1D(degree=1)` para ajustar os mesmos dados e compare os resultados.

## 2) Ajustando um modelo Polinomial: Escolha um ajustador corretamente 


Para nosso segundo exemplo, vamos ajustar um polinõmio de grau maior que 1. Neste  caso, vamos criar dados mascarados para fazer o ajuste. Note que nós adicionamos um ruido gaussiano para os dados com a função `np.random.normal(0,2)` onde é dado um numero aleatório da distribuição gaussiana  com media 0 e desvio padrão 2.

In [ ]:
N = 100
x1 = np.linspace(0, 4, N)  # Makes an array from 0 to 4 of N elements
y1 = x1**3 - 6*x1**2 + 12*x1 - 9 
# Agora vamos adicionar algum ruido para estes dados
y1 += np.random.normal(0, 2, size=len(y1)) #Uma maneira  de gerar ruido aleatorio  gaussiano
sigma = 1.5
y1_err = np.ones(N)*sigma 

Vamos plotar para ver como fica: 

In [ ]:
plt.errorbar(x1, y1, yerr=y1_err,fmt='k.')
plt.xlabel('$x_1$')  
plt.ylabel('$y_1$')

Para ajustar esses dados vamos lembrar os três passos: modelar, ajustar e realizar o ajuste. 


In [ ]:
model_poly = models.Polynomial1D(degree=3)
fitter_poly = fitting.LinearLSQFitter() 
best_fit_poly = fitter_poly(model_poly, x1, y1, weights = 1.0/y1_err**2)

In [ ]:
print(best_fit_poly)

O que aconteceria se usássemos um ajustador (método) diferente? Vamos usar o mesmo modelo, mas com  `SimplexLSQFitter` como ajustador.

In [ ]:
fitter_poly_2 = fitting.SimplexLSQFitter()
best_fit_poly_2 = fitter_poly_2(model_poly, x1, y1, weights = 1.0/y1_err**2)

In [ ]:
print(best_fit_poly_2)

Note que nós recebemos um aviso após usar `SimplexLSQFitter` para ajustar os dados. A primeira linha nos diz:

`WARNING: Modelo é linear nos parÂmetros; considere usando metodos de ajuste linear. [astropy.modeling.fitting]`

Se Nós olharmos para o modelo que escolhemos : $y = c_0 + c_1\times x + c_2\times x^2 + c_3\times x^3$, Isto é linear inos parametros $c_i$. O aviso nos traz que  `SimplexLSQFitter` trabalha  melhor com modelos que não são lineares nos parametros, e que precisamos usar um ajustador linear como `LinearLSQFitter`. A segunda linha nos diz:

`WARNING: The fit may be unsuccessful; Maximum number of iterations reached. [astropy.modeling.optimizers]`

Portanto, não é de surpreender que os resultados sejam diferentes, porque isso significa que o montador não está funcionando corretamente. Vamos discutir um método de escolha entre ajustes e lembre-se de  **prestar atenção** quando se escolhe o **ajustador**.

#### Compare os resultados

Um meio de verificar onde os parâmetros  são os melhores ajustados é calculando o [o valor quadratico do Chi Reduzido](https://en.wikipedia.org/wiki/Reduced_chi-squared_statistic). Vamos definir uma função para fazer porque vamos usá-la várias vezes.

In [ ]:
def calc_reduced_chi_square(fit, x, y, yerr, N, n_free):
    '''
    fit (array) values for the fit
    x,y,yerr (arrays) data
    N total number of points
    n_free number of parameters we are fitting
    '''
    return 1.0/(N-n_free)*sum(((fit - y)/yerr)**2)

In [ ]:
reduced_chi_squared = calc_reduced_chi_square(best_fit_poly(x1), x1, y1, y1_err, N, 4)
print('Reduced Chi Squared with LinearLSQFitter: {}'.format(reduced_chi_squared))

In [ ]:
reduced_chi_squared = calc_reduced_chi_square(best_fit_poly_2(x1), x1, y1, y1_err, N, 4)
print('Reduced Chi Squared with SimplexLSQFitter: {}'.format(reduced_chi_squared))

Como podemos ver, o *Quadrado Reduzido do Chi* para o primeiro ajuste é fechado para um , onde significa que este ajuste é melhor. Note que isso é o que nós esperamos após a discussão dos avisos.

Nós podemos então comparar dois ajustes visualmente:

In [ ]:
plt.errorbar(x1, y1, yerr=y1_err,fmt='k.')
plt.plot(x1, best_fit_poly(x1), color='r', linewidth=3, label='LinearLSQFitter()')  
plt.plot(x1, best_fit_poly_2(x1), color='g', linewidth=3, label='SimplexLSQFitter()')
plt.xlabel(r'$\log_{10}$(Period [days])')
plt.ylabel('Ks')
plt.legend()

Os resultados são os esperados, o ajuste realizado com o filtro linear é melhor que o segundo, não linear. 

**Conclusão:** Preste atenção quando escolher o ajustador.

## 3) Ajustando  uma Gaussiana: Vamos  comparar para o  scipy

Scipy tem uma função [scipy.optimize.curve_fit](https://docs.scipy.org/doc/scipy-1.0.0/reference/generated/scipy.optimize.curve_fit.html) para ajustar um meio similar que que nós fazemos. Vamos comparar os dois metodos com dados mascarados na forma de uma gaussiana. 

In [ ]:
mu, sigma, amplitude = 0.0, 10.0, 10.0
N2 = 100
x2 = np.linspace(-30, 30, N)
y2 = amplitude * np.exp(-(x2-mu)**2 / (2*sigma**2))
y2 = np.array([y_point + np.random.normal(0, 1) for y_point in y2])   #Another way to add random gaussian noise
sigma = 1
y2_err = np.ones(N)*sigma

In [ ]:
plt.errorbar(x2, y2, yerr=y2_err, fmt='k.')
plt.xlabel('$x_2$')
plt.ylabel('$y_2$')

Vamos fazer nossos três passos para fazer os ajustes que nós queremos . Para este ajuste, nós vamos usar um ajustador não linear, `LevMarLSQFitter`, porque o modelo que nós precisamaos (`Gaussian1D`) é não linear nos parâmetros. 

In [ ]:
model_gauss = models.Gaussian1D()
fitter_gauss = fitting.LevMarLSQFitter()
best_fit_gauss = fitter_gauss(model_gauss, x2, y2, weights=1/y2_err**2)

In [ ]:
print(best_fit_gauss)

Vamos tomar a [matriz de covariãncia](http://mathworld.wolfram.com/CovarianceMatrix.html) do`LevMarLSQFitter`, o qual fornece um erro para nosso ajuste  dos parâmetros pelo fazer do ajuste. `fitter.fit_info['param_cov']`. Os  elementos na diagonal desta matriz são os quadrados dos erros. Nós podemos verificara ordem dos parâmetros:

In [ ]:
model_gauss.param_names

In [ ]:
cov_diag = np.diag(fitter_gauss.fit_info['param_cov'])
print(cov_diag)

Então:

In [ ]:
print('Amplitude: {} +\- {}'.format(best_fit_gauss.amplitude.value, np.sqrt(cov_diag[0])))
print('Mean: {} +\- {}'.format(best_fit_gauss.mean.value, np.sqrt(cov_diag[1])))
print('Standard Deviation: {} +\- {}'.format(best_fit_gauss.stddev.value, np.sqrt(cov_diag[2])))

Nós podemos aplicar o mesmso método com `scipy.optimize.curve_fit`, e compare os resultados usando de novo o *valor quadratico reduzido do Chi*.

In [ ]:
def f(x,a,b,c):
    return a * np.exp(-(x-b)**2/(2.0*c**2))

In [ ]:
p_opt, p_cov = scipy.optimize.curve_fit(f,x2, y2, sigma=y1_err)
a,b,c = p_opt
best_fit_gauss_2 = f(x2,a,b,c)

In [ ]:
print(p_opt)

In [ ]:
print('Amplitude: {} +\- {}'.format(p_opt[0], np.sqrt(p_cov[0,0])))
print('Mean: {} +\- {}'.format(p_opt[1], np.sqrt(p_cov[1,1])))
print('Standard Deviation: {} +\- {}'.format(p_opt[2], np.sqrt(p_cov[2,2])))

#### Compare os resultados

In [ ]:
reduced_chi_squared = calc_reduced_chi_square(best_fit_gauss(x2), x2, y2, y2_err, N2, 3)
print('Reduced Chi Squared using astropy.modeling: {}'.format(reduced_chi_squared))

In [ ]:
reduced_chi_squared = calc_reduced_chi_square(best_fit_gauss_2, x2, y2, y2_err, N2, 3)
print('Reduced Chi Squared using scipy: {}'.format(reduced_chi_squared))

Como podemos ver, há uma diferença muito pequena no *Quadrado Reduzidp do Chi*. Nisto realmente precisava acontecer, porque o pacote `astropy.modeling` usa scipy para ajustar. A vantagem de usar `astropy.modeling` é somente precisar mudar o nome do ajustador e o modelo execute um diferente ajuste, onde o scipy nos requere lembrar a expressão da função que queremos usar.

In [ ]:
plt.errorbar(x2, y2, yerr=y2_err, fmt='k.')
plt.plot(x2, best_fit_gauss(x2), 'g-', linewidth=6, label='astropy.modeling')
plt.plot(x2, best_fit_gauss_2, 'r-', linewidth=2, label='scipy')
plt.xlabel('$x_2$')
plt.ylabel('$y_2$')
plt.legend()

**Conclusão:** Escolha o método mais conveniente para todo caso que precise ajustar. Nós recomendamos `astropy.modeling` porque é facilitador para escrever o nome da função que queremos ajustar para relembrar a expressão todo tempo que queremos usar. Então, `astropy.modeling` torna-se útil com modelos mais complicados como [duas gaussianas](http://docs.astropy.org/en/stable/modeling/#compound-models) mais um [corpo negro](http://docs.astropy.org/en/stable/modeling/#blackbody-radiation), mas isso é outro tutorial.

## Revendo:

vamos rever a conclusão que nós tomamos neste tutorial:

1. Você pode ajustar os dados com **três linhas de codigo**:
    * modelo
    * ajustador
    * Execução de ajuste aos dados 
    
    
2. **Preste atenção** quando você escolher o **ajustador**.

3. Escolha o metodo mais conveniente para todo caso que precise ajustar. nós recomendamos `astropy.modeling` para fazer **rápidos ajustes de funções conhecidas**.

## 4) Exercício: Sua vez de escolher

Para os proximos  dados:
 * Escolha o modelo e ajustador para ajustar estes dados.
 * Compare diderentes opções.

In [ ]:
N3 = 100
x3 = np.linspace(0, 3, N3)
y3 = 5.0 * np.sin(2 * np.pi * x3)
y3 = np.array([y_point + np.random.normal(0, 1) for y_point in y3])
sigma = 1.5
y3_err = np.ones(N)*sigma 

In [ ]:
plt.errorbar(x3, y3, yerr=y3_err, fmt='k.')
plt.xlabel('$x_3$')
plt.ylabel('$y_3$')